In [1]:
import sys
sys.path.insert(0, '/Users/matthewashman/github/MasterProject2018')

# Import necessary modules. Set settings. Import data.
import math
import numpy as np
import pandas as pd
import random
import pywt
import matplotlib.pyplot as plt
from statsmodels.robust import mad
from tsfresh.feature_extraction import feature_calculators
from FeatureExtraction.feature_tools import detect_peaks
from IPython.display import display, clear_output, HTML

import pdb

plt.style.use('default')

X = pd.read_pickle('/Users/matthewashman/github/MasterProject2018/Data/extracted_segments.pkl')

/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [5]:
X_S2 = X[X['S1/S2']=='S2']
X_S1 = X[X['S1/S2']=='S1']

Let's look at only feature vectors of the response to the shortest S1/S2 interval first.

In [13]:
temp = []
types = X_S2['Type'].unique()
for patient_type in types:
    patients = X_S2[X_S2['Type']==patient_type]['Patient'].unique()
    for patient in patients:
        patient_s2 = X_S2[(X_S2['Type']==patient_type) & (X_S2['Patient']==patient)]
        patient_cis = patient_s2['Coupling Interval'].unique()
        shortest_ci = np.sort(patient_cis)[0]
        shortest_ci_s2 = patient_s2[patient_s2['Coupling Interval']==shortest_ci]
        temp.append(shortest_ci_s2)

In [14]:
shortest_S2 = pd.concat(temp)
shortest_S2.head()

,Channel,Coupling Interval,Data,Patient,S1/S2,Type
99,CS1-2,230,"[-811, -639, -649, -716, -672, -732, -707, -68...",1,S2,af
102,CS3-4,230,"[-11, 49, 16, 45, 31, -14, -34, -48, -85, -79,...",1,S2,af
105,CS5-6,230,"[-181, -333, -765, -1010, -1271, -1570, -2082,...",1,S2,af
243,CS1-2,250,"[36, 30, 74, 208, 81, 232, 312, 314, 331, 235,...",2,S2,af
246,CS3-4,250,"[1453, 1119, 97, -144, -174, -30, 216, 251, 34...",2,S2,af


In [15]:
feature_list = []


# Extract features
for i, row in shortest_S2.iterrows():
    clear_output(wait=True)
    display('Extracting Features: ' + str(round(100*i/shortest_S2.index[-1],3)) + '%')

    # Get typical response for this patient and channel
    # Bad apples
    if (((row['Type'] + row['Patient']) == 'af8') & (row['Channel'] == 'CS5-6')):
        typical_response = X_S1[(X_S1['Type']==row['Type']) & 
                           (X_S1['Patient']==row['Patient']) &
                           (X_S1['Channel']==row['Channel'])
                           ].sort_values(by=['Coupling Interval'], ascending=False).iloc[2]
    elif (((row['Type'] + row['Patient']) == 'at1') & (row['Channel'] == 'CS1-2')):
        typical_response = X_S1[(X_S1['Type']==row['Type']) & 
                           (X_S1['Patient']==row['Patient']) &
                           (X_S1['Channel']==row['Channel'])
                           ].sort_values(by=['Coupling Interval'], ascending=False).iloc[4]
    else:
        typical_response = X_S1[(X_S1['Type']==row['Type']) & 
                               (X_S1['Patient']==row['Patient']) &
                               (X_S1['Channel']==row['Channel'])
                               ].sort_values(by=['Coupling Interval'], ascending=False).iloc[0]

    # Normalise responses.
    s2_response = row['Data']/max(abs(typical_response['Data']))
    s1_response = typical_response['Data']/max(abs(typical_response['Data']))
    
    typical_feature_dict = get_good_feature_dict(s1_response)
    feature_dict = get_good_feature_dict(s2_response)

    # Normalise by subtracting 'typical' feature values
    for k, v in feature_dict.items():
        feature_dict[k] = v - typical_feature_dict[k]

    feature_dict['DTW'] = dtw_around_main_energy(s2_response, s1_response)
    # Fill in the other column values
    for col, value in row.iteritems():
        feature_dict[col] = value

    feature_list.append(feature_dict)

'Extracting Features: 100.0%'

In [22]:
final_features = pd.DataFrame(feature_list)
final_features['AF'] = (final_features['Type']=='af')

In [23]:
final_features.head()

,Approximate Entropy: m=3 r=0.7,Channel,Conduction Delay: set_thresh=False,Coupling Interval,Data,Index Mass Quantile: q=0.6,Location of Maximum Energy: M=14,Number of Peaks: set_thresh=False,Patient,Percentage Fractionation: thresh=0.01,Power Spectral Entropy,Ratio Beyond 1xSTD,S1/S2,Sample Entropy Around Max Energy: width=60 r=0.025,Type,"Width of Maximum Energy: M=14, width_thresh=0.2",AF
0,-0.010016,CS1-2,4,230,"[-811, -639, -649, -716, -672, -732, -707, -68...",0.038462,5,1,1,1.538462,-0.120612,0.007692,S2,-0.002441,af,10,True
1,0.001857,CS3-4,6,230,"[-11, 49, 16, 45, 31, -14, -34, -48, -85, -79,...",0.069231,4,2,1,4.615385,0.244815,0.030769,S2,0.149385,af,2,True
2,0.208801,CS5-6,19,230,"[-181, -333, -765, -1010, -1271, -1570, -2082,...",0.230769,33,6,1,23.076923,0.017853,0.161538,S2,0.252575,af,47,True
3,0.001569,CS1-2,10,250,"[36, 30, 74, 208, 81, 232, 312, 314, 331, 235,...",0.092308,5,1,2,6.153846,-0.607676,0.030769,S2,0.348953,af,7,True
4,0.039725,CS3-4,13,250,"[1453, 1119, 97, -144, -174, -30, 216, 251, 34...",0.092308,11,1,2,0.000000,-0.102304,0.007692,S2,0.472515,af,0,True


In [31]:
import seaborn as sns
import re

%matplotlib qt
feature_names = final_features.drop(['Patient', 'Type', 'Channel', 'AF', 'S1/S2', 'Coupling Interval', 'Data'], axis=1).columns

sns.set(style="whitegrid")
# Make plot for each feature
fig, axes = plt.subplots(ncols=2, nrows=math.ceil(len(feature_names)/2), figsize=(16,9))
plt.rcParams.update({'font.size': 6})
for i,feature in enumerate(feature_names):
    if (i < math.ceil(len(feature_names)/2)):
        axes[i,0] = sns.violinplot(x='Channel', y=feature, hue='AF', split=True, palette='Set2', data=final_features, ax=axes[i,0])
        feature_ylabel = re.sub(r'(:)', r'\1\n', feature)
        axes[i,0].set_ylabel(ylabel=feature_ylabel, fontsize=8)
        axes[i,0].set(xlabel='')
    else:
        j = i - math.ceil(len(feature_names)/2)
        axes[j,1] = sns.violinplot(x='Channel', y=feature, hue='AF', split=True, palette='Set2', data=final_features, ax=axes[j,1])
        feature_ylabel = re.sub(r'(:)', r'\1\n', feature)
        axes[j,1].set_ylabel(ylabel=feature_ylabel, fontsize=8)
        axes[j,1].set(xlabel='')
        
plt.show()

In [2]:
# A shitty conduction delay detector
def get_delay(x, amp_thresh=None, set_thresh=False):
    if (set_thresh==True):
        if any(abs(x)>amp_thresh):
            return np.argmax(abs(x)>amp_thresh)
        else:
            return len(x)
    else:    
        return np.argmax(abs(x)>(max(abs(x))/2))
    
def denoise(x):
    # Obtain Daubechies N=6 wavelet coefficients
    waveletCoefs = pywt.wavedec(x, 'db7', mode='per')

    # Throw away coefficients corresponding to noise
    sigma = mad(waveletCoefs[-1])
    uThresh = 1*sigma*np.sqrt(2*np.log(len(x)))
    denoised = waveletCoefs[:]
    denoised[1:] = (pywt._thresholding.hard(i, value=uThresh) for i in denoised[1:])

    # Reconstruct the original signal
    xDenoised = pywt.waverec(denoised, 'db7', mode='per')

    return xDenoised

def get_peaks(x, height_thresh, scale_amp=None, set_scale=False, plot = False):
    x = np.array(x)
    
    # Get height_thresh
    if set_scale:
        height_thresh = height_thresh*scale_amp
    else:
        height_thresh = height_thresh*max(abs(x))
    
    # Denoise x
    xdn = denoise(x)

    # Detect peaks using detect_peaks
    pos_peak_idx = detect_peaks(xdn, mph=height_thresh, threshold = 0)
    neg_peak_idx = detect_peaks((-xdn), mph=height_thresh, threshold = 0)
    peak_idx = np.concatenate([pos_peak_idx, neg_peak_idx])
    peak_idx = np.sort(peak_idx)
    # Edge indeces aren't detected
    peak_idx = peak_idx[(peak_idx != 0) & (peak_idx != (len(xdn)-1))]

    new_peak_idx = []
    peak_amp = []
    if (len(peak_idx) > 0):
        new_peak_idx.append(peak_idx[0])
        mp_thresh = 0.2*max(abs(x))
        for i in range(len(peak_idx)-1):
            idx = peak_idx[i]
            idx_next = peak_idx[i+1]
            mid_point = int((idx_next+idx)/2)
            if (max([abs(x[idx_next]-x[mid_point]), abs(x[idx]-x[mid_point])]) > mp_thresh):
                new_peak_idx.append(idx_next)

        peak_idx = np.array(new_peak_idx)
        peak_amp = x[peak_idx]

    if plot == True:
        fig, [ax1] = plt.subplots(nrows=1, ncols=1, sharex=True, figsize=(8,8))
        ax1.plot(x, 'b' , xdn, 'r--', peak_idx, peak_amp, 'kx')
        #plt.title(fileName)
        ax1.set_xlabel('Sample')
        ax1.set_ylabel('Normalised amplitude')
        ax1.legend(['Original segment', 'Denoised segment', 'Detected peaks'])

        plt.draw()
        plt.waitforbuttonpress(0) # this will wait for indefinite time
        plt.close(fig)


    return peak_idx, peak_amp

def sample_entropy(U, m, r):

    def _maxdist(x_i, x_j):
        result = max([abs(ua-va) for ua, va in zip(x_i, x_j)])
        return result

    def _phi(m):
        x = np.zeros([N,m-1])
        for i in range(N-m+1):
            x[i,:] = U[i:i+m-1]

        C = 0
        for i in range(len(x)):
            for j in range(len(x)):
                if i != j:
                    if _maxdist(x[i,:], x[j,:]) <= r:
                        C = C + 1

        return C

    U = U/max(abs(U))
    N = len(U)

    return -np.log(_phi(m+1)/_phi(m))

def percentage_fractionation(x, peak_idxs, thresh=0.01, sr=1000):
    # Get peak indexes and amplitude
    peak_idx_diffs = np.diff(peak_idxs)
    frac_time = 0
    frac_time = np.sum(peak_idx_diffs[peak_idx_diffs < thresh*sr])
    prcnt_frac = (frac_time/len(x))*100
    return prcnt_frac

def get_local_sample_entropy(x, centre_idx, width, m=2, r=0.05):
    # Ensure width is odd
    if ((width%2) == 0):
        width += 1
        
    if (centre_idx < (width-1)/2):
        return sample_entropy(x[:width+1], m, r)
    elif (centre_idx > (len(x)-1-(width-1)/2)):
        return sample_entropy(x[len(x)-1-width:], m, r)
    else:
        return sample_entropy(x[int(centre_idx-(width-1)/2):int(centre_idx+(width+1)/2)], m, r)
    
def get_location_of_max_energy(x, M=14):
    v = np.ones(M)
    x_ = np.convolve(abs(x), v)
    return (np.argmax(x_) + math.floor(M/2))
        
def get_local_peaks(x, centre_idx, width=25, height_thresh=0.1):
    if ((width%2) == 0):
        width += 1
        
    if (centre_idx < (width-1)/2):
        return get_peaks(x[:width+1], height_thresh)
    elif (centre_idx > (len(x)-1-(width-1)/2)):
        return get_peaks(x[len(x)-1-width:], height_thresh)
    else:
        return get_peaks(x[int(centre_idx-(width-1)/2):int(centre_idx+(width+1)/2)], height_thresh)
    
def get_pse(x):
    x_fft = np.fft.rfft(x)
    x_P = (1/len(x_fft))*np.absolute(x_fft)**2
    x_p = x_P/sum(x_P)
    pse = np.sum([(-p*np.log2(p)) for p in x_p])
    if pse == np.nan:
        pdb.set_trace()
        print('WTF')
    return pse

def get_local_pse(x, centre_idx, width=50):
    if ((width%2) == 0):
        width += 1
        
    if (centre_idx < (width-1)/2):
        return get_pse(x[:width+1])
    elif (centre_idx > (len(x)-1-(width-1)/2)):
        return get_pse(x[len(x)-1-width:])
    else:
        return get_pse(x[int(centre_idx-(width-1)/2):int(centre_idx+(width+1)/2)])
    
def get_spectral_centroid(x):
    x_fft = np.fft.rfft(x)
    x_spectrum = np.absolute(x_fft)
    normalized_spectrum = x_spectrum/sum(x_spectrum)
    normalized_frequencies = np.arange(0, len(x_spectrum), 1)
    return sum(normalized_frequencies * normalized_spectrum)

def get_local_spectral_centroid(x, centre_idx, width=50):
    if ((width%2) == 0):
        width += 1
        
    if (centre_idx < (width-1)/2):
        return get_spectral_centroid(x[:width+1])
    elif (centre_idx > (len(x)-1-(width-1)/2)):
        return get_spectral_centroid(x[len(x)-1-width:])
    else:
        return get_spectral_centroid(x[int(centre_idx-(width-1)/2):int(centre_idx+(width+1)/2)])
    
def get_local_energy(x, centre_idx, width=60):
    if ((width%2) == 0):
        width += 1
        
    if (centre_idx < (width-1)/2):
        return np.sum(x[:width+1]**2)
    elif (centre_idx > (len(x)-1-(width-1)/2)):
        return np.sum(x[len(x)-1-width:]**2)
    else:
        return np.sum(x[int(centre_idx-(width-1)/2):int(centre_idx+(width+1)/2)]**2)
    
def get_width_max_energy(x, M=14, width_thresh=0.2):
    v = np.ones(M)
    x_ = np.convolve(abs(x), v)
    if any(x_[np.argmax(x_):] < width_thresh*np.max(x_)):
        end_idx = np.argmax(x_) + np.argmax(x_[np.argmax(x_):] < width_thresh*np.max(x_))
    else:
        end_idx = len(x_)-1
    if any(x_[np.argmax(x_)::-1] < width_thresh*np.max(x_)):  
        start_idx = np.argmax(x_) - np.argmax(x_[np.argmax(x_)::-1] < width_thresh*np.max(x_))
    else:
        start_idx = 0
    return (end_idx - start_idx)

def dtw_around_main_energy(s2, s1, width=12):
    s1_me = get_location_of_max_energy(s1)
    s2_me = get_location_of_max_energy(s2)
    
    if s1_me < width:
        s1_seg = s1[:(2*width+1)]
    else:
        s1_seg = s1[(s1_me-width):(s1_me+width)]
    
    if s2_me < width:
        s2_seg = s2[:(2*width+1)]
    else:
        s2_seg = s2[(s2_me-width):(s2_me+width)]
        
    fdtw = fastdtw.dtw(s2_seg, s1_seg)
    
    return fd
    s1_ = np.convolve(abs(s1), v)
    if any(s1_[np.argmax(s1_):] < width_thresh*np.max(s1_)):
        end_idx = np.argmax(s1_) + np.argmax(s1_[np.argmax(s1_):] < width_thresh*np.max(s1_))
    else:
        end_idx = len(s1_)-1
    if any(s1_[np.argmax(s1_)::-1] < width_thresh*np.max(s1_)):  
        start_idx = np.argmax(s1_) - np.argmax(s1_[np.argmax(s1_)::-1] < width_thresh*np.max(s1_))
    else:
        start_idx = 0

In [3]:
def get_good_feature_dict(x, col_prefix=''):
    feature_dict = {}
    height_thresh=0.2
    
#     feature_dict[col_prefix + 'Maximum Absolute Value'] = max(abs(x))
    
    # Hand engineered features
#     x = x/max(abs(x))
    feature_dict[col_prefix + 'Conduction Delay: set_thresh=False'] = get_delay(x)
    peaks = get_peaks(x, height_thresh)
    feature_dict[col_prefix + 'Number of Peaks: set_thresh=False'] = len(peaks[0])
    feature_dict[col_prefix + 'Percentage Fractionation: thresh=0.01'] = percentage_fractionation(x, peaks[0], thresh=0.01)
    
    # Denoise x for remaining features
    x = denoise(x)
    max_energy_idx = get_location_of_max_energy(x)
    feature_dict[col_prefix + 'Location of Maximum Energy: M=14'] = max_energy_idx
    feature_dict[col_prefix + 'Sample Entropy Around Max Energy: width=60 r=0.025'] = get_local_sample_entropy(x, max_energy_idx, 60, m=2, r=0.025)
    feature_dict[col_prefix + 'Width of Maximum Energy: M=14, width_thresh=0.2'] = get_width_max_energy(x, M=14, width_thresh=0.2)
    
    # Temporal features
    feature_dict[col_prefix + 'Approximate Entropy: m=3 r=0.7'] = feature_calculators.approximate_entropy(x, 3, 0.7)
    imq = feature_calculators.index_mass_quantile(x, [{'q': 0.6}])
    feature_dict[col_prefix + 'Index Mass Quantile: q=0.6'] = imq[0][1]
    feature_dict[col_prefix + 'Ratio Beyond 1xSTD'] = feature_calculators.ratio_beyond_r_sigma(x, 1)
    
    # Spectral features
    feature_dict[col_prefix + 'Power Spectral Entropy'] = get_pse(x)
    
    return feature_dict
    

def get_hand_engineered_feature_dict(x, thresh_cd=None, set_thresh_cd=False, thresh_peaks=None, set_thresh_peaks=False, show_peaks=False, col_prefix = ''):
    feature_dict = {}
    sf = max(abs(x))
    x = x/max(abs(x))

    # Hand engineered features
    if set_thresh_cd:
        thresh_cd = thresh_cd/sf
        feature_dict[col_prefix + 'Conduction Delay: set_thresh=True'] = get_delay(x, thresh_cd, set_thresh_cd)
        feature_dict[col_prefix + 'Conduction Delay: set_thresh=False'] = get_delay(x)
    else:
        feature_dict[col_prefix + 'Conduction Delay: set_thresh=False'] = get_delay(x)
    
    height_thresh=0.1
    if set_thresh_peaks:
        thresh_peaks = thresh_peaks/sf
        peaks = get_peaks(x, height_thresh, thresh_peaks, set_thresh_peaks, plot=False)
        feature_dict[col_prefix + 'Number of Peaks: set_thresh=True'] = len(peaks[0])
        peaks = get_peaks(x, height_thresh)
        feature_dict[col_prefix + 'Number of Peaks: set_thresh=False'] = len(peaks[0])
    else:
        peaks = get_peaks(x, height_thresh)
        feature_dict[col_prefix + 'Number of Peaks: set_thresh=False'] = len(peaks[0])
    
    peaks = get_peaks(x, height_thresh)
    feature_dict[col_prefix + 'Percentage Fractionation: thresh=0.01'] = percentage_fractionation(x, peaks[0], thresh=0.01)
    
    # Denoise x for remaining features
    x = denoise(x)
    
    max_energy_idx = get_location_of_max_energy(x)
    feature_dict[col_prefix + 'Location of Maximum Energy: M=14'] = max_energy_idx
    feature_dict[col_prefix + 'Sample Entropy Around Max Energy: width=60 r=0.025'] = get_local_sample_entropy(x, max_energy_idx, 60, m=2, r=0.025)
    feature_dict[col_prefix + 'Energy Around Max Energy'] = get_local_energy(x, max_energy_idx, 60)
    min_idx = np.argmin(x)
    max_idx = np.argmax(x)
    feature_dict[col_prefix + 'Peaks Between Min and Max'] = len([i for i in peaks[0] if ((i > min_idx) & (i < max_idx))])
    feature_dict[col_prefix + 'Width of Maximum Energy: M=14, width_thresh=0.4'] = get_width_max_energy(x, M=14, width_thresh=0.4)
    feature_dict[col_prefix + 'Width of Maximum Energy: M=14, width_thresh=0.2'] = get_width_max_energy(x, M=14, width_thresh=0.2)

    return feature_dict

def get_spectral_feature_dict(x, col_prefix = ''):
    feature_dict = {}
    # Denoise and normalise x for remaining features
    x = denoise(x)
    x = x/max(abs(x))
    
    feature_dict[col_prefix + 'Power Spectral Entropy'] = get_pse(x)
    feature_dict[col_prefix + 'Spectral Centroid'] = get_spectral_centroid(x)
    max_energy_idx = get_location_of_max_energy(x)
    feature_dict[col_prefix + 'Power Spectral Entropy Around Maximum Energy: width=30'] = get_local_pse(x, max_energy_idx, width=30)
    feature_dict[col_prefix + 'Spectral Centroid Around Maximum Energy: width=30'] = get_local_spectral_centroid(x, max_energy_idx, width=30)
    feature_dict[col_prefix + 'Power Spectral Entropy Around Maximum Energy: width=60'] = get_local_pse(x, max_energy_idx, width=60)
    feature_dict[col_prefix + 'Spectral Centroid Around Maximum Energy: width=60'] = get_local_spectral_centroid(x, max_energy_idx, width=60)
    
    return feature_dict
    
def get_temporal_feature_dict(x, col_prefix = ''):

    feature_dict = {}
    feature_dict[col_prefix + 'Maximum Absolute Value'] = np.max(abs(x))
    
    # Denoise and normalise x for remaining features
    x = denoise(x)
    x = x/max(abs(x))


    erbc = feature_calculators.energy_ratio_by_chunks(x, [{'num_segments':10, 'segment_focus':3}, {'num_segments':10, 'segment_focus':2}])
    feature_dict[col_prefix + 'Energy Ratio by Chunks: num_segments=10 segment_focus=2'] = erbc[1][1]
    feature_dict[col_prefix + 'Energy Ratio by Chunks: num_segments=10 segment_focus=3'] = erbc[0][1]
    feature_dict[col_prefix + 'Approximate Entropy: m=3 r=0.7'] = feature_calculators.approximate_entropy(x, 3, 0.7)
    feature_dict[col_prefix + 'Ratio Beyond 5xSTD'] = feature_calculators.ratio_beyond_r_sigma(x, 5)
    feature_dict[col_prefix + 'Ratio Beyond 4xSTD'] = feature_calculators.ratio_beyond_r_sigma(x, 4)
    feature_dict[col_prefix + 'Ratio Beyond 3xSTD'] = feature_calculators.ratio_beyond_r_sigma(x, 3)
    feature_dict[col_prefix + 'Ratio Beyond 2xSTD'] = feature_calculators.ratio_beyond_r_sigma(x, 2)
    feature_dict[col_prefix + 'Ratio Beyond 1xSTD'] = feature_calculators.ratio_beyond_r_sigma(x, 1)
    # A fraction q of the mass lies to the left of i. (Alternative to conduction delay?)
    imq = feature_calculators.index_mass_quantile(x, [{'q': 0.6}, {'q': 0.4}])
    feature_dict[col_prefix + 'Index Mass Quantile: q=0.6'] = imq[0][1]
    feature_dict[col_prefix + 'Index Mass Quantile: q=0.4'] = imq[1][1]
    

    return feature_dict